In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
%pip install dask dask-expr

Note: you may need to restart the kernel to use updated packages.


In [4]:
import dask.dataframe as dd

In [5]:
train = dd.read_csv("/home/owner/Documents/DEV/graphs/graphs_and_structural_analysis/Santander_Product_Recommendation/train_ver2.csv",
                        dtype={'cod_prov': 'float64',
                                    'indrel_1mes': 'object',
                                    'ult_fec_cli_1t': 'object',
                                    'conyuemp': 'object',
                                    'age': 'object',
                                    'age': 'object',
                                    'antiguedad': 'object',
                                    'ind_actividad_cliente': 'float64',
                                    'ind_nom_pens_ult1': 'float64',
                                    'ind_nomina_ult1': 'float64',
                                    'ind_nuevo': 'float64',
                                    'indrel': 'float64',
                                    'tipodom': 'float64'
                                },
            )

In [6]:
train = train.compute()

/home/owner/Documents/DEV/graphs/graphs_and_structural_analysis/.venv/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (12,16,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


# Предобработка, которую я перенес

# A теперь положим в нормальный файл

In [7]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [8]:
train.to_feather("train.feather")